In [ ]:
#pip install --user m2crypto

In [1]:
from geni.aggregate import FrameworkRegistry
from geni.aggregate.context import Context
from geni.aggregate.user import User

def buildContext ():
  framework = FrameworkRegistry.get("portal")()
  framework.cert = "/home/lngo/.cert/cloudlab.pem"
  framework.key = "/home/lngo/.cert/cloudlab.pem"

  user = User()
  user.name = "lngo"
  user.urn = "urn:publicid:IDN+ch.geni.net+user+lngo"
  user.addKey("/home/lngo/.ssh/id_rsa.pub")

  context = Context()
  context.addUser(user, default = True)
  context.cf = framework
  context.project = "BDSLab"

  return context

context = buildContext()
import geni.aggregate.instageni as IG
import pprint
pprint.pprint(IG.GPO.getversion(context))

ImportError: /lib64/libc.so.6: version `GLIBC_2.14' not found (required by /home/lngo/.local/lib/python2.7/site-packages/M2Crypto/__m2crypto.so)

In [11]:
!module purge; module add anaconda/4.3.0; python cloudlab-geni-examples/singleXenVM.py

/bin/sh: line 0: unset: `spark-submit': not a valid identifier
/bin/sh: line 0: unset: `spark-sql': not a valid identifier
/bin/sh: line 0: unset: `spark-shell': not a valid identifier
Traceback (most recent call last):
  File "cloudlab-geni-examples/singleXenVM.py", line 16, in <module>
    node = request.XenVM("node")
AttributeError: 'Request' object has no attribute 'XenVM'
Exception TypeError: TypeError("'NoneType' object is not callable",) in <bound method TempfileManager.__del__ of <geni.tempfile.TempfileManager object at 0x2b5035ceba10>> ignored


In [2]:
import geni.portal as portal
import geni.rspec.pg as rspec

request = rspec.Request()
node = rspec.RawPC

In [3]:
import geni.portal as portal
import geni.rspec.pg as rspec

request = rspec.Request()
node1 = request.XenVM("node1")
iface1 = node1.addInterface("if1")

# Specify the component id and the IPv4 address
iface1.component_id = "eth1"
iface1.addAddress(rspec.IPv4Address("192.168.1.1", "255.255.255.0"))

node2 = request.XenVM("node2")
iface2 = node2.addInterface("if2")

# Specify the component id and the IPv4 address
iface2.component_id = "eth2"
iface2.addAddress(rspec.IPv4Address("192.168.1.2", "255.255.255.0"))

link = request.LAN("lan")

link.addInterface(iface1)
link.addInterface(iface2)

portal.context.printRequestRSpec()

AttributeError: 'Request' object has no attribute 'XenVM'

In [ ]:
"""An example of constructing a profile with a single Xen VM.

Instructions:
Wait for the profile instance to start, and then log in to the VM via the
ssh port specified below.  (Note that in this case, you will need to access
the VM through a high port on the physical host, since we have not requested
a public IP address for the VM itself.)
"""

import geni.portal as portal
import geni.rspec.pg as rspec

# Create a Request object to start building the RSpec.
request = portal.context.makeRequestRSpec()
 
# Create a XenVM
node = request.XenVM("node")

# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec()

In [9]:
%%writefile hpccsystems.py
import geni.portal as portal
import geni.rspec.pg as rspec

# Create a Request object to start building the RSpec.
request = portal.context.makeRequestRSpec()
 
# Create a link with type LAN
link = request.LAN("lan")

# Generate the nodes
for i in range(4):
    node = request.RawPC("node" + str(i))
    node.disk_image = "urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"
    iface = node.addInterface("if" + str(i))
    iface.component_id = "eth1"
    iface.addAddress(rspec.IPv4Address("192.168.1." + str(i + 1), "255.255.255.0"))
    link.addInterface(iface)
    
    node.addService(rspec.Execute(shell="bash",
                                  command="sudo adduser --ingroup admin --disabled-password hpcc"))
    node.addService(rspec.Execute(shell="bash",
                                  command="sudo adduser hpcc sudo"))
    node.addService(rspec.Execute(shell="bash",
                                  command="sudo apt-get update"))
    node.addService(rspec.Execute(shell="bash",
                                  command="sudo wget http://cdn.hpccsystems.com/releases/CE-Candidate-5.2.2/bin/platform/hpccsystems-platform-community_5.2.2-1trusty_amd64.deb"))
    node.addService(rspec.Execute(shell="bash",
                                  command="sudo dpkg -i hpccsystems-platform-community_5.2.2-1trusty_amd64.deb"))
    node.addService(rspec.Execute(shell="bash",
                                  command="sudo apt-get -y -f install"))

# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec()

Overwriting hpccsystems.py


In [10]:
!module load anaconda/4.3.0; python hpccsystems.py

<rspec xmlns:client="http://www.protogeni.net/resources/rspec/ext/client/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.geni.net/resources/rspec/3" xsi:schemaLocation="http://www.geni.net/resources/rspec/3 http://www.geni.net/resources/rspec/3/request.xsd" type="request">
  <link client_id="lan">
    <interface_ref client_id="node0:if0"/>
    <interface_ref client_id="node1:if1"/>
    <interface_ref client_id="node2:if2"/>
    <interface_ref client_id="node3:if3"/>
    <link_type name="lan"/>
  </link>
  <node client_id="node0" exclusive="true">
    <sliver_type name="raw">
      <disk_image name="urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"/>
    </sliver_type>
    <interface client_id="node0:if0" component_id="eth1">
      <ip address="192.168.1.1" netmask="255.255.255.0" type="ipv4"/>
    </interface>
    <services>
      <execute shell="bash" command="sudo adduser --ingroup admin --disabled-password hpcc"/>
      <execute shell="bash" co